### Solves the highly nonlinear Richard's equation in 1D using the mixed form approach and Newton's method to find the roots F(x) = 0

In [1]:
import numpy as np
import scipy.sparse as sp
import scipy.optimize as op
from core import *

In [2]:
import matplotlib.pyplot as plt
from matplotlib import animation, rcParams
from IPython.display import HTML

In [3]:
plt.rcParams['animation.html'] = 'html5'

In [4]:
# Spatial and temporal discretization
k = 4
m = 60
a = 0
b = 40
dx = (b - a) / m
t = 360.
dt = 1.
n = int(t / dt)

# Problem's parameters (from Michael Celia's paper on Unsaturated Flow)
# https://doi.org/10.1029/WR026i007p01483
alpha = 1.611e+6
theta_s = 0.287
theta_r = 0.075
theta_g = theta_s - theta_r
beta = 3.96
K_s = 0.00944
A = 1.175e+6
gamma = 4.74
ic = -61.5
bot_bc = -20
top_bc = -61.5

# Get mimetic operators
D = div1D(k, m, dx)
G = grad1D(k, m, dx)
I = interpol1D(m, 0.5)

psi_init = np.ones(m) * ic
psi_old = np.append(np.insert(psi_init, 0, ic), ic)

def K_psi(psi):
    return (K_s * A) / (A + np.power(np.absolute(psi), gamma))

def theta_psi(psi):
    return ((alpha * theta_g) / (alpha + np.power(np.absolute(psi), beta))) + theta_r

def F(psi):
    psi_new = np.append(np.insert(psi, 0, bot_bc), top_bc)
    K = I @ K_psi(psi_new)
    
    theta_t = (theta_psi(psi_new) - theta_psi(psi_old)) / dt
    
    d1 = -(D @ (np.diag(K) @ (G @ psi_new)))
    d1 = np.append(np.insert(d1[1:-1], 0, bot_bc), top_bc)
    
    Dz = D @ K
    Dz = np.append(np.insert(Dz[1:-1], 0, bot_bc), top_bc)
        
    fval = theta_t + d1 + Dz
    return fval[1:-1]

/Users/nutrik/Documents/Science/scodes/Python/pymole/core/div1D.py:53: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  D[i, j:j+k] = coeffs
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/div1D.py:73: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  A[i, 0:q] = coeffs
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/div1D.py:79: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  D[1:p+1, 0:q] = A
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/div1D.py:95: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  D[n_rows-p-1:n_rows-1, n_cols-q:n_cols] = A
/Users/nutrik/Documents/Science/scodes/Python/pymole/core/grad1D.py:52: SparseEfficiencyWarning: Changing the sparsity structure of a csr_

In [5]:
xgrid = np.append(np.insert(np.arange(dx/2., b, dx), 0, 0), b)

In [8]:
def animate_richards(solution):
    fig = plt.figure(figsize=(8,6))
    ax = plt.gca()
    ax.set_xlim((0, 40))
    ax.set_ylim((-70, 10))
    ax.set_xlabel('Depth')
    ax.set_ylabel('Pressure head')
    plt.grid(True)
    line, = ax.plot([], [], lw=2)
    plt.close()

    def animate(i):
        line.set_data(xgrid, solution[:,i])
        ax.set_title('Richards Eqn. (Mixed form) solved with pyMOLE\nt = {:01.2f}s'.format(i * dt))
        return line,

    return animation.FuncAnimation(fig, animate, frames=solution.shape[1], interval=50, blit=True)

In [7]:
solution = np.zeros((psi_old.shape[0], n))

# Time integration loop
for i in range(n):
    if i==0:
        init_guess = np.ones(m) * ic
    else:
        init_guess = sol.x
    # Find the roots using Newton's method
    sol = op.root(F, init_guess, method='lm')
    psi_old = np.append(np.insert(sol.x, 0, bot_bc), top_bc)
    solution[..., i] = psi_old.copy()

In [10]:
anim = animate_richards(solution)

In [11]:
anim